# Model AI terkait Stunting akibat Malnutrisi dan lainnya di Kab. Mamberamo Raya

Kita akan mencoba melakukan pemodelan dengan XGBoost model

# Inisialisasi Package

In [ ]:
# General Package
import io
import logging
import warnings
import pandas as pd
import numpy as np
from copy import deepcopy
from google.colab import drive

warnings.filterwarnings('ignore')
pd.options.display.float_format = '{:,.2f}'.format
np.random.seed(1)

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline
#sns.set_style('darkgrid')
plt.style.use('fivethirtyeight')
plt.rcParams['lines.linewidth'] = 1.5
dark_style = {
    'figure.facecolor': '#212946',
    'axes.facecolor': '#212946',
    'savefig.facecolor':'#212946',
    'axes.grid': True,
    'axes.grid.which': 'both',
    'axes.spines.left': False,
    'axes.spines.right': False,
    'axes.spines.top': False,
    'axes.spines.bottom': False,
    'grid.color': '#2A3459',
    'grid.linewidth': '1',
    'text.color': '0.9',
    'axes.labelcolor': '0.9',
    'xtick.color': '0.9',
    'ytick.color': '0.9',
    'font.size': 12 }
plt.rcParams.update(dark_style)

from pylab import rcParams
rcParams['figure.figsize'] = (12,6)

# Fungsi Visualisasi

In [ ]:
from bokeh.models import HoverTool, ColumnDataSource, NumeralTickFormatter, DatetimeTickFormatter
from bokeh.io import curdoc, output_notebook
from bokeh.themes import built_in_themes
from bokeh.plotting import figure, show
from bokeh.palettes import gray
from bokeh.colors import RGB

output_notebook()
curdoc().theme = built_in_themes['caliber']

In [ ]:
def BokehPlot_1D(data: pd.DataFrame,
                 dates: str = 'date',
                 original: str = 'suhu',
                 Title: str = 'Grafik Suhu Harian',
                 Color: str = '#0a8234',
                 date_unit: str = 'day', # Opsi Argumen: 'year', 'month', 'day'
                ) -> None:
    output_notebook()
    curdoc().theme = built_in_themes['caliber']

    yourdata = data.copy()
    if dates not in yourdata.columns:
        yourdata[dates] = yourdata.index


    # Grafik inisialisasi
    Grafik = figure(title=Title,
                    background_fill_color=RGB(242, 242, 242),
                    background_fill_alpha=0.85,
                    )
    Grafik.width = 1200
    Grafik.height = 400

    # Modifikasi Data
    if date_unit == 'month':
        yourdata['Penanggalan'] = pd.to_datetime(yourdata[dates], format='%Y-%m', errors='coerce')
        Grafik.xaxis.formatter = DatetimeTickFormatter(months="%b %Y")
        x_axis_label = 'Bulan'
        hover_format = "@tanggal{%b %Y}"
    elif date_unit == 'year':
        yourdata['Penanggalan'] = pd.to_datetime(yourdata[dates], format='%Y', errors='coerce')
        Grafik.xaxis.formatter = DatetimeTickFormatter(years="%Y")
        x_axis_label = 'Tahun'
        hover_format = "@tanggal{%Y}"
    else:
        yourdata['Penanggalan'] = pd.to_datetime(yourdata[dates], format='%Y-%m-%d', errors='coerce')
        Grafik.xaxis.formatter = DatetimeTickFormatter(days="%d %b %Y")
        x_axis_label = 'Harian'
        hover_format = "@tanggal{%d %b %Y}"

    Grafik.xaxis.axis_label = x_axis_label
    Grafik.yaxis.axis_label = original

    # Join dataset ke Source
    yourdata.dropna(subset=['Penanggalan'], inplace=True)
    source = ColumnDataSource(data=dict(
        tanggal=yourdata['Penanggalan'],
        Original=yourdata[original],
    ))

    Grafik.line(x='tanggal', y='Original',
                source=source, legend_label=original.upper(),
                line_width=2, color=Color)

    # Hover setting
    Tipo = HoverTool()
    Tipo.tooltips = [(x_axis_label, hover_format),
                     ("Origin",   "@Original{0,.2f}"),
                    ]
    Tipo.formatters = {'@tanggal': 'datetime'}
    Tipo.mode = 'vline'
    Grafik.add_tools(Tipo)

    # Customize plot
    Grafik.title.align = 'center'
    Grafik.title.text_font_size = '20pt'
    Grafik.title.text_color = '#d11950'
    Grafik.legend.location = 'top_left'
    Grafik.legend.click_policy = 'hide'

    Grafik.yaxis.formatter = NumeralTickFormatter(format="0,0")
    show(Grafik)

In [ ]:
def Bokeh_Decomposition(DataX : np.ndarray,
                        DataY : np.ndarray,
                        Judul : str = 'Komponen Trend',
                        color : str = "#d11950",
                       ) -> None :
    Data = ColumnDataSource(data=dict(
                              x = DataX,
                              y = DataY,
                            ))

    graphDeretWaktu = figure(width        = 1200,
                             height       = 250,
                             x_axis_type  = "datetime",
                             x_axis_label = "Tanggal",
                             y_axis_label = "Nilai",
                             background_fill_color = gray(20)[17],
                             background_fill_alpha = 0.85,
                            )

    graphDeretWaktu.line('x', 'y',
                         source     = Data,
                         color      = color,
                         line_width = 1,
                        )
    graphDeretWaktu.title.text           = Judul
    graphDeretWaktu.title.text_font_size = "20pt"
    graphDeretWaktu.title.align          = "center"
    graphDeretWaktu.title.text_color     = color
    graphDeretWaktu.yaxis[0].formatter = NumeralTickFormatter(format="0,0")

    THeHover = HoverTool(tooltips=[
                        ("Tanggal", "@x{%F}"),
                        ("Nilai", "@y{0,.2f}")],
                        formatters={"@x": "datetime"
                        })
    graphDeretWaktu.add_tools(THeHover)
    show(graphDeretWaktu)


In [ ]:
def BokehPlot(mydata      : pd.DataFrame,
              dates       : str = 'date',
              original    : str = 'Close',
              prediksi    : str = 'Forecasting',
              batas_atas  : str = 'AutoARIMA-hi-95',
              batas_bawah : str = 'AutoARIMA-lo-95',
              modelname   : str = 'ARIMA',
              title       : str = '',
             ) -> None :

    output_notebook()
    curdoc().theme = built_in_themes['caliber']

    yourdata = mydata.copy()
    if dates not in yourdata.columns:
        yourdata[dates] = yourdata.index

    if batas_atas not in yourdata.columns:
        data_dict = {
            'tanggal': yourdata[dates],
            'Original': yourdata[original],
            'Predict': yourdata[prediksi]
        }
        varea = False
    else:
        data_dict = {
            'tanggal': yourdata[dates],
            'Original': yourdata[original],
            'Predict': yourdata[prediksi],
            'upper':yourdata[batas_atas],
            'lower':yourdata[batas_bawah],
        }
        varea = True
    DataSumber = ColumnDataSource(data=data_dict)

    if len(str(title)) <= 5 :
        title = f'Plot {modelname} dalam data Order Demand'
    Grafik = figure(title= title,
                    x_axis_type='datetime',
                    x_axis_label='Tanggal',
                    y_axis_label='Besar Order Demand',
                    background_fill_color=gray(20)[17],
                    background_fill_alpha=0.85,
                    )
    Grafik.width = 1200
    Grafik.height = 400

    Grafik.line(x='tanggal', y='Original',
                source=DataSumber, legend_label='Besar Order Demand pada Gudang',
                line_width=2, color='#252392')
    Grafik.line(x='tanggal', y='Predict',
                source=DataSumber, legend_label=f'{modelname}',
                line_width=2, color='#d40d4c')
    if varea :
        Grafik.varea(x='tanggal',
                     y1='lower',
                     y2='upper',
                     source=DataSumber,
                     color='#61e041',
                     alpha=0.7,
                     legend_label='95% CI'
                    )

    Tipo = HoverTool()
    Tipo.tooltips = [("Tanggal", "@tanggal{%F}"),
                     ("Origin", "@Original{0,.2f}"),
                     (modelname, "@Predict{0,.2f}"),
                     ]
    Tipo.formatters = {'@tanggal': 'datetime'}
    Grafik.add_tools(Tipo)

    Grafik.title.align = 'center'
    Grafik.title.text_font_size = '20pt'
    Grafik.title.text_color = '#d11950'
    Grafik.legend.location = 'top_left'
    Grafik.legend.click_policy = 'hide'
    Grafik.yaxis.formatter = NumeralTickFormatter(format="0,0")

    show(Grafik)

# Inisialisasi Data

In [ ]:
drive.mount('/content/Jack', force_remount = True)

Mounted at /content/Jack


In [ ]:
import os
import zipfile as zp

def ZipExtract(zpath : str,
               paswd : str,
               where : str,
              ) -> None :
    try:
        with zp.ZipFile(zpath, 'r') as f:
            f.setpassword(paswd.encode('utf-8'))
            f.extractall(where)
        print(f"File '{zpath}' berhasil diekstrak.")
        print('Berikut list directory-nya:')
        print(os.listdir(where))
        return os.path.realpath(os.path.join(where, 'StuntingData.xlsx'))
    except zp.BadZipFile:
        print(f"Error: File '{zpath}' bukan file ZIP yang valid atau rusak.")
    except RuntimeError as Arch:
        if "Bad password" in str(Arch):
            print(f"Error: Kata sandi salah untuk file '{zpath}'.")
        else:
            print(f"Error saat mengekstrak file: {Arch}")
    except FileNotFoundError:
        print(f"Error: File '{zpath}' tidak ditemukan.")
    except Exception as Arch:
        print(f"Terjadi kesalahan tak terduga: {Arch}")

In [ ]:
Zippath = '/content/Jack/MyDrive/LECTURE/PAPER 01/StuntingData.zip'
#Excelpath = ZipExtract(Zippath, '*******', './')

In [ ]:
ExcelPath = '/content/Jack/MyDrive/LECTURE/PAPER 01/StuntingData.xlsx'
Data = pd.read_excel(ExcelPath)

In [ ]:
Data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 607 entries, 0 to 606
Data columns (total 11 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   No                607 non-null    int64 
 1   DistrikKecamatan  607 non-null    object
 2   Kec_Kode          607 non-null    int64 
 3   Y                 607 non-null    int64 
 4   X1                607 non-null    int64 
 5   X2                607 non-null    int64 
 6   X3                607 non-null    int64 
 7   X4                607 non-null    int64 
 8   X5                607 non-null    int64 
 9   X6                607 non-null    int64 
 10  X7                607 non-null    int64 
dtypes: int64(10), object(1)
memory usage: 52.3+ KB


Berikut ini adalah detail dari setiap fitur pada data kita.

| Variable | Nama Variable | Skala| Kategori | Definisi Operasional |
|---|---|---|---|---|
| Y| Stunting| Nominal| 1 = Stunting 0 = Tidak Stunting | Kondisi gagal tumbuh pada anak balita akibat kekurangan gizi kronis yang terjadi dalam jangka waktu panjang, terutama pada 1.000 hari pertama kehidupan (sejak dalam kandungan hingga usia dua tahun).|
| X1 | Jenis kelamin | Nominal| 1 = Laki-laki 0 = Perempuan | Suatu ciri biologis yang membedakan antara laki-laki dan perempuan yang ditentukan sejak lahir.|
| X2 | Level Status Gizi Buruk menurut WHO | Nominal | 0 - 6 dengan 6 sangat bergizi buruk | Status gizi menurut WHO adalah suatu keadaan yang diakibatkan oleh keseimbangan antara asupan gizi dari makanan dengan kebutuhan nutrisi yang diperlukan tubuh untuk metabolism. |
| X3 | ASI Eksklusif | Nominal| 1 = Tidak ASI Eksklusif 0 = ASI Eksklusif | Pemberian hanya Air Susu Ibu kepada bayi sejak lahir hingga berusia 6 bulan tanpa tambahan makanan/minuman lain (kecuali obat/vitamin sesuai anjuran medis).|
| X4 | Kebiasaan Merokok Dalam Keluarga | Nominal| 1 = Ada Kebiasaan Merokok 0 = Tidak Ada Kebiasaan Merokok | Kondisi di mana terdapat satu atau lebih anggota keluarga yang secara rutin merokok, baik di dalam maupun di sekitar lingkungan rumah tempat anak tinggal. |
| X5 | Akses Air Bersih| Nominal| 1 = Tidak Ada Akses Air Bersih 0 = Ada Akses Air Bersih | Kondisi ketersediaan air yang layak dikonsumsi/digunakan sehari-hari. Air bersih berasal dari sumber memenuhi syarat kesehatan (PDAM, sumur/mata air terlindung, air hujan yang dikelola). |
| X6 | Jamban Sehat| Nominal| 1 = Tidak Ada Jamban Sehat 0 = Ada Jamban Sehat | Fasilitas sanitasi memenuhi syarat kesehatan: punya leher angsa (water seal), penutup, saluran tertutup, tidak mencemari air. |
| X7 | Wilayah Sulit | Nominal| 1 = Sulit 0 = Tidak Sulit | Kategori geografis menggambarkan daerah dengan keterbatasan akses infrastruktur dasar, baik karena kondisi geografis berat, terbatas transportasi, atau jauh dari pusat pelayanan publik. |

# Penggunaan Data pada DuckDB

In [ ]:
import duckdb

PathDB = '/content/Jack/MyDrive/Malnutrition_Paper.db'
DuckConnect = duckdb.connect(database = PathDB)